In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
import datetime
from langchain.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

In [3]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert researcher.
            Current Time: {time}
            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Recommend search queries to research information and improve your answer
            (NOTE: You MUST provide the search queries at all costs)"""
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
actor_prompt_template = actor_prompt_template.partial(time=lambda:datetime.datetime.now().isoformat())

In [4]:
from schemas import AnswerQuestion


llm = ChatOpenAI()
parser_json = JsonOutputToolsParser(return_id=True)
parser_pydantic = PydanticToolsParser(tools=[AnswerQuestion])

In [5]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer"
)

In [6]:
first_responder = first_responder_prompt_template | llm.bind_tools(
    tools=[AnswerQuestion], tool_choice="AnswerQuestion"
)

In [7]:
human_message = HumanMessage(
    content="""Write about AI-Powered SOC / autonomous SOC problem domain
    List startups that do that and raised capital"""
)

In [8]:
chain = (
    first_responder_prompt_template 
    | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
    | parser_pydantic
)

In [10]:
res = chain.invoke(input={"messages": [human_message]})

In [11]:
for inst in res[0]:
    print(inst[0], end=": ")
    print(inst[1])
    print()
    print()
    print()

answer: The domain of AI-Powered Security Operations Centers (SOC) or Autonomous SOC involves the use of artificial intelligence and machine learning algorithms to enhance cybersecurity capabilities. These advanced systems can autonomously detect, analyze, and respond to security incidents in real-time, reducing the burden on human analysts and improving the overall security posture of an organization. AI-powered SOCs can handle a vast amount of data efficiently, identify patterns, and detect anomalies that may indicate potential threats, thus enabling proactive threat mitigation.

Some startups that operate in this domain and have raised significant capital include Darktrace, which has pioneered AI-powered cybersecurity with its Autonomous Response technology; Vectra, a leader in AI-based threat detection and response; and Cynet, known for its autonomous breach protection platform. These startups have attracted substantial investments due to the increasing demand for advanced cybersec

In [12]:
revise_instructions = """Revise your previous answer using the new information.
You should use the previous critique to add important information to your answer.
You MUST include numerical citations in your revised answer to ensure it can be verified.
Add a "References" section to the bottom of your answer (which does not count towards the word limit).
In the form of:
- [1] https://example.com
- [2] https://example.com
You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words"""

In [14]:
from schemas import ReviseAnswer

revisor_prompt_template = actor_prompt_template.partial(first_instruction=revise_instructions)
revisor = (
    revisor_prompt_template
    | llm.bind_tools(tools=[ReviseAnswer], tool_choice="ReviseAnswer")
)